In [26]:
import ollama   
import pandas as pd
from tqdm import tqdm  # 导入进度条库

# 调用 Ollama 模型进行分析
def analyze_abstract_with_ollama(prompt):
    response = ollama.chat(
        model="llama3.2:latest",
        stream=False,
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": 0}
    )
    if "message" in response and "content" in response["message"]:
        return response["message"]["content"]  # 返回有效内容
    else:
        return "未返回有效内容"

# 生成提示词
def generate_prompt(abstract):
    prompt = f"""
    You are a research assistant specialized in analyzing scientific papers related to high-entropy materials and their application in hydrogen evolution reaction (HER) catalysis.
    Your task is to extract the following specific information from the research article abstract provided below:

    1. Article Type: Determine whether this is a research article or a review article.
    2. identify if it discusses relevant to OER catalysis.
    3. list the elements used (e.g., Co, Ni, Fe) specifically for catalysis.
    
    Provide only the requested information, keeping the response concise and clear.

    Abstract: {abstract}
    """
    return prompt

# 解析分析结果的函数
def parse_analysis_result(analysis_result, is_research, is_high_entropy):
    return {"Model_Response": analysis_result} if is_research and is_high_entropy else {"Model_Response": analysis_result}

# 处理文献
def process_documents(documents):
    results = []
    for doc in tqdm(documents, desc="处理文献", unit="篇"):  # 显示进度条
        prompt = generate_prompt(doc["Abstract"])  # 生成提示词
        analysis_result = analyze_abstract_with_ollama(prompt)  # 调用模型分析
        
        # 解析返回的结果并判断类型
        is_research = "research article" in analysis_result.lower()  # 判断是否为研究性文章
        is_high_entropy = "high-entropy" in analysis_result.lower()  # 判断是否涉及高熵材料
        parsed_result = parse_analysis_result(analysis_result, is_research, is_high_entropy)  # 解析结果
        
        results.append({
            "标题": doc["Article Title"],  # 添加标题
            "摘要": doc["Abstract"],  # 添加摘要  
            "模型响应": parsed_result["Model_Response"]  # 添加模型响应
        })
    return results

# 保存结果到 CSV 文件
def save_results_to_csv(results, output_file):
    df = pd.DataFrame(results)  # 将结果转换为 DataFrame
    df.to_csv(output_file, index=False, encoding='utf-8-sig')  # 保存为 CSV 文件，支持中文字符

# 读取文献文件（假设文献格式为 XLS 文件）
def read_documents(file_path):
    df = pd.read_excel(file_path)  # 读取 XLS 文件
    documents = df[['Article Title', 'Abstract']].to_dict(orient='records')  # 获取指定列
    return documents

# 主流程
if __name__ == "__main__":
    input_file = 'OER.xls'  # 输入文件名
    documents = read_documents(input_file)  # 读取文献
    results = process_documents(documents)  # 处理文献
    
    # 生成输出文件名
    output_file = f"{input_file.split('.')[0]}_ollama文献分析结果.csv"  
    save_results_to_csv(results, output_file)  # 保存结果到 CSV 文件

    print(f"文献分析已完成，结果已保存到 {output_file}")  # 输出完成信息



处理文献: 100%|██████████| 973/973 [15:54<00:00,  1.02篇/s]

文献分析已完成，结果已保存到 OER_ollama文献分析结果.csv


In [19]:
#输入是txt文本

import ollama  
import csv
import pandas as pd
from tqdm import tqdm  # 导入进度条库

# 调用 Ollama 模型进行分析
def analyze_abstract_with_ollama(prompt):
    response = ollama.chat(
        model="llama3.2:latest",
        stream=False,
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": 0}
    )
    if "message" in response and "content" in response["message"]:
        return response["message"]["content"]  # 返回有效内容
    else:
        return "未返回有效内容"

# 生成提示词
def generate_prompt(abstract):
    prompt = f"""
    You are a research assistant specialized in analyzing scientific papers related to high-entropy materials and their application in hydrogen evolution reaction (HER) catalysis.
    Your task is to extract the following specific information from the research article abstract provided below:

    1. Article Type: Determine whether this is a research article or a review article.
    2. identify if it discusses relevant to OER catalysis.
    3. list the high-entropy elements used (e.g., Co, Ni, Fe) specifically for catalysis.
    
    Provide only the requested information, keeping the response concise and clear.

    Abstract: {abstract}
    """
    return prompt

# 解析分析结果的函数
def parse_analysis_result(analysis_result, is_research, is_high_entropy):
    return {"Model_Response": analysis_result} if is_research and is_high_entropy else {"Model_Response": analysis_result}

# 处理文献
def process_documents(documents):
    results = []
    for doc in tqdm(documents, desc="处理文献", unit="篇"):  # 显示进度条
        prompt = generate_prompt(doc["abstract"])  # 生成提示词
        analysis_result = analyze_abstract_with_ollama(prompt)  # 调用模型分析
        
        # 解析返回的结果并判断类型
        is_research = "research article" in analysis_result.lower()  # 判断是否为研究性文章
        is_high_entropy = "high-entropy" in analysis_result.lower()  # 判断是否涉及高熵材料
        parsed_result = parse_analysis_result(analysis_result, is_research, is_high_entropy)  # 解析结果
        
        results.append({
            "标题": doc["title"],  # 添加标题
            "摘要": doc["abstract"],  # 添加摘要  
            "模型响应": parsed_result["Model_Response"]  # 添加模型响应
        })
    return results

# 保存结果到 CSV 文件
def save_results_to_csv(results, output_file):
    df = pd.DataFrame(results)  # 将结果转换为 DataFrame
    df.to_csv(output_file, index=False, encoding='utf-8-sig')  # 保存为 CSV 文件，支持中文字符

# 读取文献文件（假设文献格式为 "标题\t摘要"）
def read_documents(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')  # 使用制表符分隔
        for row in reader:
            title, abstract = row  # 获取标题和摘要
            documents.append({"title": title, "abstract": abstract})  # 添加到文献列表
    return documents

# 主流程
if __name__ == "__main__":
    documents = read_documents('OER.txt')  # 读取文献
    results = process_documents(documents)  # 处理文献
    save_results_to_csv(results, 'HER_ollama文献分析结果.csv')  # 保存结果到 CSV 文件

    print("文献分析已完成，结果已保存到 OER_ollama文献分析结果.csv")  # 输出完成信息


ValueError: too many values to unpack (expected 2)